In [ ]:
# gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# cv.imshow('gray image', gray)
# gray = cv.equalizeHist(gray)

# # Threshold (adaptive or Otsu)
# thresh = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, 
#                                cv.THRESH_BINARY_INV, 11, 2)

# # Morphological operations to clean noise
# kernel = cv.getStructuringElement(cv.MORPH_RECT, (2,2))
# cv.imshow('thresholded image', thresh)

# morph = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel)
# cv.imshow('morphological image', morph)

# # Invert the binary image so characters are white on black
# inverted = cv.bitwise_not(morph)

# # Find contours (now characters are white)
# contours, _ = cv.findContours(inverted, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
# cv.imshow('contoured image', contours)

# # Extract and save each character bounding box
# for i, cnt in enumerate(contours):
#     x, y, w, h = cv.boundingRect(cnt)
#     if w > 10 and h > 10:  # Filter small noise
#         char_img = inverted[y:y+h, x:x+w]
#         cv.imwrite(f'results/char_{i}.png', char_img)

# cv.waitKey(0)
# cv.destroyAllWindows()

In [ ]:
#How to train the model:
import tensorflow_datasets as tfds
import tensorflow as tf

# Load EMNIST (digits + letters)
ds_train = tfds.load('emnist/byclass', split='train', as_supervised=True)
ds_test = tfds.load('emnist/byclass', split='test', as_supervised=True)

# Normalize images
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

ds_train = ds_train.map(normalize_img).batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(normalize_img).batch(128).prefetch(tf.data.AUTOTUNE)

# Simple CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(62, activation='softmax')  # 62 classes (0–9, A–Z, a–z)
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(ds_train, validation_data=ds_test, epochs=5)


